# **DATA CLEANING**

## Objectives

* Evaluate Missing data
* Cleaning data

## Inputs

* outputs/datasets/collection/house_price_records.csv

## Outputs

* outputs/datasets/collection/cleaned_house_price_records.csv

## Additional Comments

* ```EnclosedPorch``` and ```WoodDeckSF``` columns have about 90% data missing. Consider to drop them for analysis.

* Imputate missing data

* Create data cleaning pipeline


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/pp5_project_heritage_housing/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/pp5_project_heritage_housing'

Load necessary modules

In [4]:
# plotting lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# for data cleaning
from sklearn.pipeline import Pipeline

---

# Data Cleaning

## Drop features with lot of missing data

As discussed in Sale Price Study notebook, we will drop two columns  ```EnclosedPorch``` and ```WoodDeckSF``` columns have about 90% data missing.

In [8]:
# Make sure to work on a copy of data
df_source_data=pd.read_csv("outputs/datasets/collection/house_price_records.csv")
df=df_source_data.copy()
df.head(5)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,961,NaN,NaN,No,216,ALQ,540,NaN,642,Unf,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,RFn,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000


**Split Train and Test Set**

In [10]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

TrainSet shape: (1168, 24) 
TestSet shape: (292, 24)


In [14]:
# Drop two variables
variables_method = ['EnclosedPorch','WoodDeckSF']

from feature_engine.selection import DropFeatures
imputer = DropFeatures(features_to_drop=variables_method)
imputer.fit(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

We don't need to evaluate distribution effect on dropped features as it is columns we are dropping, not rows. So, let's move forward

---

## Imputation for other missing data

We have concluded the following so far from previous workbook:


- ```LotFrontage```has 17.7 % missing values. However, ydata detailed profile report shows that it is  correlated with ```LotArea```, It also makes sense in physical world. Bigger the lot area, the higher the chances that LotFrontage (i.e. linear feet of street connected to property) is bigger. So, we can leverage this relation for missing data imputaion.

- ```GarageFinish```has 11.1% missing values. However, ydata detailed profile report shows that it is  correlated with  ```GarageArea```. In phsyical world, they are independent. For example, a bigger garden does not nessarily mean high qality finish. So, we may have to look at other areas to find imputation strategies, for ex. distribution.

- ```BsmtFinType1```has 7.8% missing values. However, ydata detailed profile report shows that it is  correlated with  ```BsmtExposure```. In physical world, "walkout or garden level walls exposure level" and "quality of basement" are independent. So, perhaps this correlation is just coinsidence. So, we may have to look at other strategies for data impuration, for ex. distribution.

- ```BedroomAbvGr```has 6.8% missing values. However, It seems to be highly correlated with ```2ndFlrSF```and ```GrLivArea```. In physical world it does not make sense. Bedroom above ground has no relation with 2nd floor surface area and living area. So, we may have to look at other strategies for data impuration, for ex. distribution.

- ```GarageYrBlt```has 5.5% missing values. However, ydata detailed profile report shows that it is  correlated with  ```GarageArea``` , ```SalePrice```and ```YearRemodAdd```. In physical world, it may be possible that with time (GarageYrBlt), larger and larger (or smaller and smaller) garages were built. It is possible that with newer the garage, better the sales price. We can explore these relationships further for missing data imputation. 

If we address all of the above, it will cover almost 50% of total missing values and bring the missing value percentage at dataset level from 9.8% to nearly 5%. This is acceptable to work with for prediction purpose.

Now,
- As you can see there are two vaiables [LotFrontage,GarageYrBlt] where we can leverage regression to explore relationship with other variables and use that to fill missing data. That is multi variate data imputation.

- Feaure-engine currently supports univariate imputation stategies (Source: https://feature-engine.trainindata.com/en/latest/api_doc/imputation/index.html). For multi-variate imputation stategies, we need to use iterative imputer from Scikit-learn.

- This will broaden the scope of project, hence for now I decide to use univariate imputation strategies for all the variables listed above.

---

### Univariate Imputation using Feature Engine

**Since we are modifiying rows, we need to create a custom function that will help evaluate the data cleaning effect on distribution of these variables**

In [15]:
# Adopted from Unit 9 of feature engine section

def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):
  """
  Function to visualize data cleaning effect
  """
  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over variables, 
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1